## Case info extraction

In [18]:
import os
import pydicom
import shutil
from tqdm import tqdm

input_roots = [r'H:\202211', r'H:\202212', r'H:\Knee_MRI_For_Label', 
               r'H:\XGJ_batch_6', r'I:\GE', r'I:\GEPACS', r'I:\影像下载']

output_root = r'G:\MR_Knee'
unique_case_list = []
case_info = dict()
cols = ['pid', 'name','gender','age', 'study_date','study_time','study_desc', 'modality','manufacturer','model','strength','location']
for col in cols:
    case_info.setdefault(col, [])


number = 0
for input_root in tqdm(input_roots):

    for root, folders, files in os.walk(input_root):
        for file in files:
            try:
                ds = pydicom.dcmread(os.path.join(root, file))
            except Exception as e:
                continue
            try:
                pid = str(ds.PatientID)
                study_date = str(ds.StudyDate)
                study_time = str(ds.StudyTime)
            except Exception as e:
                continue
            try:
                name = str(ds.PatientName)
            except Exception as e:
                name = 'NA'
            uid = name + '_' + study_date + study_time

            try:
                age = str(ds.PatientAge)
            except Exception as e:
                age = 'NA'
            try:
                gender = str(ds.PatientSex)
            except Exception as e:
                gender = 'NA'
            try:
                study_desc = str(ds.StudyDescription)
            except Exception as e:
                study_desc = 'NA'
            try:
                modality = str(ds.Modalilty)
                if not modality == 'MR':
                    continue
            except Exception as e:
                modality = 'NA'
            try:
                manufacturer = str(ds.Manufacturer)
            except Exception as e:
                manufacturer = 'NA'
            try:
                model = str(ds.ManufactuerModelName)
            except Exception as e:
                model = 'NA'
            try:
                strength = str(ds.MagneticFieldStrength)
            except Exception as e:
                strength = 'NA'
            if uid not in unique_case_list:
                unique_case_list.append(uid)
                location = root
                for col in cols:
                    case_info[col].append(eval(col))
                number += 1
                if number % 50 == 0:
                    print(number)
            else:
                break

                    

100%|██████████| 7/7 [00:00<00:00, 6508.56it/s]


In [19]:
import pandas as pd

df_result = pd.DataFrame(case_info)
df_result.to_excel('knee_dicom_info2.xlsx', index=False)

In [ ]:
input_roots = [r'H:\01', r'H:\02', r'H:\03', r'H:\04', r'H:\05', r'H:\06', r'H:\0529', r'H:\2022', r'H:\202206', 'H:\202207', r'H:\202208', r'H:\202209', r'H:\202210', r'H:\202211', r'H:\202212', r'H:\Knee_MRI_For_Label', r'H:\XGJ_batch_6', r'I:\GE', r'I:\GEPACS', 'r'I:\影像下载']


SyntaxError: invalid syntax (3788261737.py, line 1)

In [ ]:
r'H:\01', r'H:\02', r'H:\03', r'H:\04', r'H:\05', r'H:\06', r'H:\0529', r'H:\2022', r'H:\202206', 'H:\202207', 
               r'H:\202208', r'H:\202209', r'H:\202210',

## case info matching

In [59]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


patient_info = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\knee_patient_info.xlsx'
dicom_info = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\knee_dicom_info.xlsx'

df_1 = pd.read_excel(patient_info, dtype=str)
df_2 = pd.read_excel(dicom_info, dtype=str)

print(df_2.head(2))



        pid          name gender   age study_date     study_time  \
0  10415085  YAO YUN LONG      M  060Y   20230112         185401   
1  10446934     ZHANG QUN      F  055Y   20230119  162303.988000   

                    study_desc modality        manufacturer model strength  \
0   MR Left Knee Joint Routine      NaN  GE MEDICAL SYSTEMS   NaN        3   
1  MR Right Knee Joint Routine      NaN             SIEMENS   NaN        3   

                                            location findings impression  
0  H:\01\10415085\5795093\1.2.840.113619.2.353.28...      NaN        NaN  
1  H:\01\10446934\1.2.840.113619.186.240311752081...      NaN        NaN  


In [63]:

for i in tqdm(range(len(df_1))):
    pid = df_1.iloc[i]['放射科号']
    study_date = df_1.iloc[i]['检查日期'].replace('-','')
    findings = df_1.iloc[i]['放射表现']
    impression = df_1.iloc[i]['放射诊断']
    df_2.loc[(df_2['pid'] == pid) & (df_2['study_date'] == study_date),'findings'] = findings
    df_2.loc[(df_2['pid'] == pid) & (df_2['study_date'] == study_date),'impression'] = impression


  0%|          | 0/12706 [00:00<?, ?it/s]

100%|██████████| 12706/12706 [03:19<00:00, 63.60it/s]


In [64]:
df_2.to_excel(r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\july.xlsx', index=False)

## remove duplicate cases

In [72]:
input_file = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\july.xlsx'

delete_index = []
unique_case = []

df = pd.read_excel(input_file)
for index, row in df.iterrows():
    pid = str(row.pid)
    study_date = str(row.study_date)
    study_time = str(row.study_time)
    id = pid + '_' + study_date+'_'+study_time
    if id not in unique_case:
        unique_case.append(id)
    else:
        delete_index.append(index)


In [73]:
print(len(delete_index))

3697


In [74]:
df2 = df.drop(delete_index)

In [75]:
df2.to_excel(r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\july2.xlsx', index=False)

## cartilage extractoin

In [81]:
df = pd.read_excel(r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\july2.xlsx')

In [87]:
cartilage = ['水肿', '软骨','囊变', '凹陷','骨赘']
result = []

for index, row in df.iterrows():
    impression = str(row.impression)
    flag = False
    for word in cartilage:
        if word in impression:
            #df.iloc[index,'cartilage'] = 1
            result.append(1)
            flag = True
            break
    if not flag:
        result.append(0)

df['cartilage'] = result



In [88]:
df.to_excel(r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\july3.xlsx', index=False)

In [96]:
import os
import pandas as pd

input_root = r'D:\1st'
cols = ['pid','category','vendor','study_date','series_number','series_desc' ]
case_info = dict()
for col in cols:
    case_info.setdefault(col, [])

catagory_list = os.listdir(input_root)
for category in catagory_list:
    vendors = os.listdir(os.path.join(input_root, category))
    for vendor in vendors:
        for folder in os.listdir(os.path.join(input_root, category, vendor)):
            pid, study_date = folder.split('_')
            series_desc = os.listdir(os.path.join(input_root, category, vendor, folder))
            series_desc = ['_'.join(x.split('_')[:-1]) for x in series_desc]
            series_desc = [x for x in series_desc if 'loc' not in x.lower()]
            series_desc = [x for x in series_desc if 'scout' not in x.lower()]
            series_number = len(series_desc)
            for col in cols:
                case_info[col].append(eval(col))
df = pd.DataFrame(case_info)
df.to_excel(r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\case_series_info.xlsx', index=False)


In [3]:
import os
import pandas as pd
import shutil

input_root = r'F:\1st'
cols = ['pid','category','vendor','study_date','series_number','series_desc' ]
case_info = dict()
for col in cols:
    case_info.setdefault(col, [])

catagory_list = os.listdir(input_root)
for category in catagory_list:
    vendors = os.listdir(os.path.join(input_root, category))
    for vendor in vendors:
        for folder in os.listdir(os.path.join(input_root, category, vendor)):
            pid, study_date = folder.split('_')
            cur_folder = os.path.join(input_root, category, vendor, folder)
            for series in os.listdir(cur_folder):
                for del_key in ['loc', 'scout','report','asset']:
                    if del_key in series.lower():
                        shutil.rmtree(os.path.join(cur_folder, series))
            series_desc = os.listdir(cur_folder)
            series_desc = ['_'.join(x.split('_')[:-1]) for x in series_desc]
            #series_desc = [x for x in series_desc if 'loc' not in x.lower()]
            #series_desc = [x for x in series_desc if 'scout' not in x.lower()]
            series_number = len(series_desc)
            for col in cols:
                case_info[col].append(eval(col))
df = pd.DataFrame(case_info)
df.to_excel(r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\03_Research Platform\04_Knee_MR\case_series_info_0809.xlsx', index=False)

    